## Data Preparation

In [2]:
import pandas as pd 
from pandas.io.json import json_normalize
import numpy as np 
from pymongo import MongoClient
import json, ast

### Database Connection

In [3]:
DB_USERNAME = 'anutibara'
DB_PASSWORD = 'anutibara'
DB_HOST = 'scraping-cluster-7dtgt.gcp.mongodb.net'
DB_NAME = 'scraping_db'
try:
    client = MongoClient(f'mongodb+srv://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}')
    print("Database connected successfully")
except Exception as e:
    print("Error to connect to database: ", e)
db = client.get_database('scraping_db')
properties = db.properties

Database connected successfully


### Database Queries

##### Total Documents

In [4]:
properties.count_documents({})

34

### Convert Mongo Collection to DataFrame

In [88]:
json_documents = list(properties.find({}))
df_general_info = pd.DataFrame(json_documents, columns=['_id', 'urlProperty', 'scrapingDate', 'modifyDate', 'code', 'status', 'type', 'use', 'nameProject', 'description'])
df_location = json_normalize(json_documents, 'location')
df_owner_property = json_normalize(json_documents, 'ownerProperty')
df_features = json_normalize(json_documents, 'features')
df_more_features = json_normalize(json_documents, 'moreFeatures')
df_offers_type = json_normalize(json_documents, 'offersType')
df = pd.concat([df_general_info, df_location, df_owner_property, df_features, df_more_features, df_offers_type], axis=1)

### Rename DataFrame Columns 

In [116]:
df.columns = [
                'Id_Mongoose', 'Id_Property', 'Scraping_Date', 'Modify_Date', 'Code', 'Active', 'Type', 'New_Property', 
                'Name_Project', 'Description', 'Address', 'City', 'Country', 'Department', 'Latitude', 'Longitude',
                'Neighborhood', 'Sector', 'Contract_Type_Owner_Property', 'Financing_Owner_Property', 'Id_Owner_Property',
                'Name_Owner_Property', 'Schedule_Owner_Property', 'Admon_Price', 'Antiquity', 'Bathrooms', 'Condition',
                'Construction_Area', 'Floors', 'Garages', 'Includes_Administration', 'Interior_Floors', 'Price', 'Private_Area',
                'Rooms', 'Square_Meters', 'Square_Meters_Price', 'Stratum', 'Weather', 'Exterior_Features', 'Interior_Features',
                'Sector_Features', 'Area_Offers_Type', 'Bathrooms_Offers_Type', 'Transaction_Offers_Type', 'Price_Offers_Type',
                'Private_Area_Offers_Type', 'Property_Offers_Type', 'Rooms_Offers_Type'
             ]

### Data Cleaning Methods

In [117]:
def clean_status(status):
    if(status == 'Active'):
        return True
    return False

def clean_use(use_status):
    if(use_status == 'Nuevo'):
        return True
    return False

## Data Cleaning

In [120]:
df['Active'] = df['Active'].apply(clean_status)
df['New_Property'] = df['New_Property'].apply(clean_use)
df.head(2)

,Id_Mongoose,Id_Property,Scraping_Date,Modify_Date,Code,Active,Type,New_Property,Name_Project,Description,...,Exterior_Features,Interior_Features,Sector_Features,Area_Offers_Type,Bathrooms_Offers_Type,Transaction_Offers_Type,Price_Offers_Type,Private_Area_Offers_Type,Property_Offers_Type,Rooms_Offers_Type
0,5db1aebc3fa8955315cf92c6,https://www.fincaraiz.com.co/bosston-lives/med...,10/24/2019 09:00:58 AM,10/9/2019 2:21:13 PM,4981342.0,False,Venta,False,BOSSTON LIVES,Boston es un sector tranquilo donde se pueden ...,...,"[Ascensor, Salón de Juegos]","[Balcón, Comedor]","[Cerca centro comercial, Cómodas vias de acces...",58.50,2,Venta,246.925.000,,Apartamento,2
1,5db1aebccc8b1ab334cf92c6,https://www.fincaraiz.com.co/apartamento-en-ve...,10/24/2019 09:00:58 AM,10/24/2019 6:17:38 AM,5024650.0,False,Venta,False,Apartamento en Venta - Medellín La América,Apartamento ubicado en la América (Medellín / ...,...,[],"[Balcón, Barra estilo americano, Calentador, C...","[Colegios / Universidades, Parques cercanos, S...",64.15,2,Venta,252.487.000,,Apartamento,3
